# 🎯 Enhanced Sleep Disorder Classification - Complete Pipeline (GOOGLE COLAB VERSION)

## 📌 Setup Instructions for Google Colab:
1. **Upload your dataset** to Google Drive
2. **Mount Google Drive** (run the cell below)
3. **Update DATA_PATH** to point to your CSV file
4. **Run all cells** in order

## 🚀 Key Improvements:
- Attention-Enhanced BiLSTM-CNN
- Advanced Data Augmentation  
- K-Fold Cross-Validation
- Model Ensemble
- Comprehensive Visualization

---

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")
print("\nYour files are now accessible at: /content/drive/MyDrive/")

## 1️⃣ Install Required Packages (if needed)

In [ ]:
# Install any missing packages
# !pip install -q tensorflow scikit-learn matplotlib seaborn pandas numpy scipy

print("✅ All packages ready!")

## 2️⃣ Setup & Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, roc_curve, auc, precision_recall_curve,
    confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input, Dense, Conv1D, MaxPooling1D, AveragePooling1D, MaxPool1D,
    Flatten, Dropout, BatchNormalization, Activation, Add, Concatenate,
    LSTM, Bidirectional, GlobalAveragePooling1D, Layer
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from scipy.interpolate import interp1d
from time import time
import json
import os

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"✅ Imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"NumPy version: {np.__version__}")

%matplotlib inline

## 3️⃣ Data Augmentation Functions

In [ ]:
# Data Augmentation Utilities

def time_warp(x, sigma=0.2, knot=4):
    """Apply time warping to the signal"""
    orig_steps = np.arange(x.shape[0])
    random_warps = np.random.normal(loc=1.0, scale=sigma, size=(knot+2,))
    warp_steps = (np.linspace(0, x.shape[0]-1, num=knot+2))
    ret = np.interp(orig_steps, warp_steps, random_warps)
    ret = ret / ret.sum() * x.shape[0]
    ret = np.cumsum(ret)
    if len(x.shape) == 1:
        return np.interp(orig_steps, ret, x)
    else:
        return np.array([np.interp(orig_steps, ret, x[:, i]) for i in range(x.shape[1])]).T

def magnitude_warp(x, sigma=0.2, knot=4):
    """Apply magnitude warping to the signal"""
    orig_steps = np.arange(x.shape[0])
    random_warps = np.random.normal(loc=1.0, scale=sigma, size=(knot+2,))
    warp_steps = (np.linspace(0, x.shape[0]-1, num=knot+2))
    warper = np.interp(orig_steps, warp_steps, random_warps)
    if len(x.shape) == 1:
        return x * warper
    else:
        return x * warper[:, np.newaxis]

def jitter(x, sigma=0.03):
    """Add random Gaussian noise"""
    noise = np.random.normal(loc=0., scale=sigma * np.std(x), size=x.shape)
    return x + noise

def scaling(x, sigma=0.1):
    """Randomly scale the signal amplitude"""
    factor = np.random.normal(loc=1., scale=sigma)
    return x * factor

def time_shift(x, shift_range=0.1):
    """Randomly shift the signal in time (circular)"""
    shift = int(np.random.uniform(-shift_range, shift_range) * x.shape[0])
    return np.roll(x, shift, axis=0)

def window_slice(x, reduce_ratio=0.9):
    """Randomly crop and resize the signal"""
    target_len = int(reduce_ratio * x.shape[0])
    if target_len >= x.shape[0]:
        return x
    start = np.random.randint(0, x.shape[0] - target_len)
    end = start + target_len
    sliced = x[start:end]
    if len(x.shape) == 1:
        return np.interp(np.arange(x.shape[0]), np.arange(len(sliced)), sliced)
    else:
        return np.array([np.interp(np.arange(x.shape[0]), np.arange(len(sliced)), sliced[:, i]) 
                        for i in range(x.shape[1])]).T

def rotation(x):
    """Randomly flip/invert the signal"""
    flip = np.random.choice([-1, 1])
    return flip * x

def augment_signal(x, augmentation_list=['jitter', 'scaling', 'time_warp', 'magnitude_warp'], n_augmentations=2):
    """Apply random augmentations to a signal"""
    augmented = x.copy()
    selected = np.random.choice(augmentation_list, size=n_augmentations, replace=False)
    for aug in selected:
        if aug == 'jitter':
            augmented = jitter(augmented)
        elif aug == 'scaling':
            augmented = scaling(augmented)
        elif aug == 'time_warp':
            augmented = time_warp(augmented)
        elif aug == 'magnitude_warp':
            augmented = magnitude_warp(augmented)
        elif aug == 'time_shift':
            augmented = time_shift(augmented)
        elif aug == 'window_slice':
            augmented = window_slice(augmented)
        elif aug == 'rotation':
            augmented = rotation(augmented)
    return augmented

def augment_dataset(X, y, augmentation_factor=2, augmentation_methods=['jitter', 'scaling', 'time_warp']):
    """Augment entire dataset"""
    X_aug_list = [X]
    y_aug_list = [y]
    for i in range(augmentation_factor):
        X_new = np.array([augment_signal(x, augmentation_methods, n_augmentations=2) for x in X])
        X_aug_list.append(X_new)
        y_aug_list.append(y)
    X_aug = np.concatenate(X_aug_list, axis=0)
    y_aug = np.concatenate(y_aug_list, axis=0)
    indices = np.random.permutation(len(X_aug))
    X_aug = X_aug[indices]
    y_aug = y_aug[indices]
    print(f"Original dataset size: {len(X)}")
    print(f"Augmented dataset size: {len(X_aug)}")
    print(f"Augmentation factor: {len(X_aug) / len(X):.2f}x")
    return X_aug, y_aug

print("✅ Data augmentation functions loaded!")

## 4️⃣ Attention Layer & Model Architecture

In [ ]:
# Custom Attention Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[-1],),
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(name='attention_context',
                                 shape=(input_shape[-1],),
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
    
    def call(self, x):
        uit = tf.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        ait = tf.tensordot(uit, self.u, axes=1)
        attention_weights = tf.nn.softmax(ait, axis=1)
        attention_weights = tf.expand_dims(attention_weights, axis=-1)
        weighted_input = x * attention_weights
        return tf.reduce_sum(weighted_input, axis=1), attention_weights
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

# Build Attention-Enhanced BiLSTM-CNN Model
def build_attention_bilstm_cnn(input_shape, use_attention=True):
    input_signal = Input(shape=input_shape, name='input')
    
    # Block 1: Initial Feature Extraction
    x = Conv1D(filters=32, kernel_size=7, strides=1, padding='same', name='conv1_1')(input_signal)
    x = BatchNormalization(name='bn1_1')(x)
    x = Activation('relu', name='relu1_1')(x)
    x = Conv1D(filters=32, kernel_size=7, strides=1, padding='same', name='conv1_2')(x)
    x = BatchNormalization(name='bn1_2')(x)
    x = Activation('relu', name='relu1_2')(x)
    skip1 = x
    
    # Block 2: Deeper Feature Extraction with Skip
    x = Conv1D(filters=32, kernel_size=9, strides=1, padding='same', name='conv2_1')(x)
    x = BatchNormalization(name='bn2_1')(x)
    x = Activation('relu', name='relu2_1')(x)
    x = Conv1D(filters=32, kernel_size=9, strides=1, padding='same', name='conv2_2')(x)
    x = BatchNormalization(name='bn2_2')(x)
    x = Add(name='skip_add_1')([x, skip1])
    x = Activation('relu', name='relu2_3')(x)
    
    # Block 3: Downsampling
    x = Conv1D(filters=64, kernel_size=9, strides=1, padding='same', name='conv3_1')(x)
    x = BatchNormalization(name='bn3_1')(x)
    x = Activation('relu', name='relu3_1')(x)
    x = MaxPooling1D(pool_size=2, padding='same', name='pool1')(x)
    x = Dropout(0.3, name='dropout1')(x)
    
    # Block 4: More CNN layers
    x = Conv1D(filters=64, kernel_size=7, strides=1, padding='same', name='conv4_1')(x)
    x = BatchNormalization(name='bn4_1')(x)
    x = Activation('relu', name='relu4_1')(x)
    x = Conv1D(filters=32, kernel_size=5, strides=1, padding='same', name='conv4_2')(x)
    x = BatchNormalization(name='bn4_2')(x)
    x = Activation('relu', name='relu4_2')(x)
    x = MaxPooling1D(pool_size=2, padding='same', name='pool2')(x)
    
    # Block 5: Bidirectional LSTM
    x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), name='bilstm1')(x)
    x = BatchNormalization(name='bn_lstm1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), name='bilstm2')(x)
    x = BatchNormalization(name='bn_lstm2')(x)
    
    # Block 6: Attention Mechanism
    if use_attention:
        x, attention_weights = AttentionLayer(name='attention')(x)
    else:
        x = GlobalAveragePooling1D(name='global_avg_pool')(x)
    
    # Block 7: Dense Classification Layers
    x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01), name='dense1')(x)
    x = Dropout(0.4, name='dropout2')(x)
    x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01), name='dense2')(x)
    x = Dropout(0.3, name='dropout3')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)
    
    model = keras.Model(inputs=input_signal, outputs=output, name='AttentionBiLSTM_CNN')
    return model

# Baseline CNN-LSTM
def build_simple_cnn_lstm(input_shape):
    model = keras.Sequential([
        Input(shape=input_shape),
        Conv1D(32, 7, activation='relu', padding='same'),
        MaxPooling1D(2),
        Conv1D(64, 5, activation='relu', padding='same'),
        MaxPooling1D(2),
        LSTM(64, return_sequences=True),
        LSTM(32, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ], name='Simple_CNN_LSTM')
    return model

print("✅ Model architectures defined!")

## 5️⃣ Visualization Functions

In [ ]:
# Visualization Utilities

def plot_training_history(history, save_path=None):
    """Plot training and validation loss/accuracy"""
    if hasattr(history, 'history'):
        history = history.history
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Plot accuracy
    axes[0].plot(history['accuracy'], 'b-', linewidth=2, label='Training Accuracy')
    axes[0].plot(history['val_accuracy'], 'r-', linewidth=2, label='Validation Accuracy')
    axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    axes[0].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
    axes[0].legend(loc='lower right', fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Plot loss
    axes[1].plot(history['loss'], 'b-', linewidth=2, label='Training Loss')
    axes[1].plot(history['val_loss'], 'r-', linewidth=2, label='Validation Loss')
    axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Loss', fontsize=12, fontweight='bold')
    axes[1].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
    axes[1].legend(loc='upper right', fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

def plot_confusion_matrix(y_true, y_pred, class_names=['Healthy', 'Unhealthy'], normalize=True, save_path=None):
    """Plot enhanced confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_display = cm_norm
        fmt = '.2%'
    else:
        cm_display = cm
        fmt = 'd'
    
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(cm_display, annot=True, fmt=fmt, cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                linewidths=2, linecolor='white', ax=ax)
    
    if normalize:
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j + 0.5, i + 0.7, f'({cm[i, j]})',
                       ha='center', va='center', fontsize=10, color='gray')
    
    ax.set_ylabel('True Label', fontsize=13, fontweight='bold')
    ax.set_xlabel('Predicted Label', fontsize=13, fontweight='bold')
    ax.set_title('Confusion Matrix', fontsize=15, fontweight='bold', pad=20)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

def plot_roc_curve(y_true, y_pred_proba, save_path=None):
    """Plot ROC curve"""
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    plt.fill_between(fpr, tpr, 0, alpha=0.2, color='orange')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
    plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=13, fontweight='bold')
    plt.legend(loc='lower right', fontsize=11)
    plt.grid(True, alpha=0.3)
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()
    return roc_auc

def generate_metrics_report(y_true, y_pred, y_pred_proba, class_names=['Healthy', 'Unhealthy']):
    """Generate comprehensive metrics report"""
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    try:
        roc_auc = roc_auc_score(y_true, y_pred_proba)
    except:
        roc_auc = 0.0
    
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else:
        specificity = 0
    
    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall (Sensitivity)': recall,
        'Specificity': specificity,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    }
    
    print("\n" + "="*70)
    print("📊 COMPREHENSIVE METRICS REPORT")
    print("="*70)
    print(f"\nTotal Samples: {len(y_true)}")
    print("\n" + "-"*70)
    print("PERFORMANCE METRICS:")
    print("-"*70)
    for metric, value in metrics_dict.items():
        print(f"  {metric:30s}: {value:.4f} ({value*100:.2f}%)")
    print("="*70)
    print("\nCLASSIFICATION REPORT:")
    print("="*70)
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    print("\nCONFUSION MATRIX:")
    print("="*70)
    print(pd.DataFrame(cm, index=class_names, columns=class_names))
    print("="*70 + "\n")
    
    return metrics_dict

print("✅ Visualization functions loaded!")

## 6️⃣ Load Dataset

**⚠️ IMPORTANT: Update DATA_PATH below to your dataset location in Google Drive**

In [ ]:
# TODO: Update this path to your dataset location in Google Drive
# Example: '/content/drive/MyDrive/ML/H&U_classification/dataset/healthy_unhealthy1.csv'
DATA_PATH = '/content/drive/MyDrive/your_folder/healthy_unhealthy1.csv'

# Load data
try:
    data = np.loadtxt(DATA_PATH, delimiter=',')
    print(f"✅ Data loaded successfully!")
    print(f"   Shape: {data.shape}")
    
    # Split features and labels
    X = data[:, 0:1024]  # First 1024 columns are features
    y = data[:, -1]      # Last column is label (0=Healthy, 1=Unhealthy)
    
    print(f"\n📊 Dataset Statistics:")
    print(f"   Total samples: {len(X)}")
    print(f"   Feature dimensions: {X.shape[1]}")
    print(f"   Healthy samples: {np.sum(y == 0)} ({np.sum(y == 0)/len(y)*100:.1f}%)")
    print(f"   Unhealthy samples: {np.sum(y == 1)} ({np.sum(y == 1)/len(y)*100:.1f}%)")
    print(f"   Class balance ratio: {np.sum(y == 0) / np.sum(y == 1):.2f}:1")
    
except FileNotFoundError:
    print("❌ Data file not found!")
    print("Please update DATA_PATH with your dataset location.")
    print(f"Current path: {DATA_PATH}")
    raise

## 7️⃣ Data Preparation & Augmentation

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42
)

print(f"📊 Data Split:")
print(f"   Training set: {len(X_train)} samples")
print(f"   Test set: {len(X_test)} samples")
print(f"   Train class distribution: {np.bincount(y_train.astype(int))}")
print(f"   Test class distribution: {np.bincount(y_test.astype(int))}")

In [ ]:
# Apply data augmentation
APPLY_AUGMENTATION = True  # Set to False to skip augmentation
AUGMENTATION_FACTOR = 1     # Increase for more augmented data

if APPLY_AUGMENTATION and AUGMENTATION_FACTOR > 0:
    print("🔄 Applying data augmentation...")
    X_train_aug, y_train_aug = augment_dataset(
        X_train, y_train,
        augmentation_factor=AUGMENTATION_FACTOR,
        augmentation_methods=['jitter', 'scaling', 'time_warp', 'magnitude_warp']
    )
else:
    print("⏭️ Skipping data augmentation")
    X_train_aug, y_train_aug = X_train, y_train

# Reshape for CNN input
X_train_aug = X_train_aug.reshape(-1, 1024, 1)
X_test_reshaped = X_test.reshape(-1, 1024, 1)

print(f"\n✅ Final training data shape: {X_train_aug.shape}")
print(f"✅ Final test data shape: {X_test_reshaped.shape}")

## 8️⃣ Build & Compile Model

In [ ]:
# Build attention model
input_shape = (1024, 1)

print("🏗️ Building Attention-Enhanced BiLSTM-CNN model...")
model = build_attention_bilstm_cnn(input_shape, use_attention=True)

# Compile model
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()
print(f"\n✅ Model built successfully!")
print(f"   Total parameters: {model.count_params():,}")

## 9️⃣ Train Model

In [ ]:
# Training configuration
EPOCHS = 150
BATCH_SIZE = 64
MODEL_SAVE_PATH = '/content/drive/MyDrive/best_attention_model.h5'

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', patience=30, verbose=1, restore_best_weights=True, mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1, mode='min'
)

checkpoint = ModelCheckpoint(
    MODEL_SAVE_PATH, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'
)

print("🚀 Starting model training...\n")
start_time = time()

history = model.fit(
    X_train_aug, y_train_aug,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1,
    shuffle=True
)

training_time = time() - start_time

print(f"\n✅ Training completed!")
print(f"   Total time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"   Model saved to: {MODEL_SAVE_PATH}")

## 🔟 Evaluate Model

In [ ]:
# Generate predictions
y_pred_proba = model.predict(X_test_reshaped, verbose=0).flatten()
y_pred = (y_pred_proba > 0.5).astype(int)

# Visualizations
plot_training_history(history)
plot_confusion_matrix(y_test, y_pred, normalize=True)
plot_roc_curve(y_test, y_pred_proba)

# Comprehensive metrics
attention_metrics = generate_metrics_report(y_test, y_pred, y_pred_proba)

## 1️⃣1️⃣ Compare with Baseline

In [ ]:
# Train baseline model
print("🔄 Training baseline CNN-LSTM model for comparison...\n")

baseline_model = build_simple_cnn_lstm((1024, 1))
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

baseline_history = baseline_model.fit(
    X_train_aug, y_train_aug,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
    verbose=0
)

print("✅ Baseline model trained!")

# Baseline predictions
y_pred_baseline_proba = baseline_model.predict(X_test_reshaped, verbose=0).flatten()
y_pred_baseline = (y_pred_baseline_proba > 0.5).astype(int)

# Calculate metrics
baseline_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_baseline),
    'F1-Score': f1_score(y_test, y_pred_baseline),
    'ROC-AUC': roc_auc_score(y_test, y_pred_baseline_proba)
}

# Create comparison
comparison = pd.DataFrame({
    'Attention-BiLSTM-CNN': [attention_metrics['Accuracy'], attention_metrics['F1-Score'], attention_metrics['ROC-AUC']],
    'Baseline-CNN-LSTM': [baseline_metrics['Accuracy'], baseline_metrics['F1-Score'], baseline_metrics['ROC-AUC']]
}, index=['Accuracy', 'F1-Score', 'ROC-AUC'])

print("\n" + "="*70)
print("📊 MODEL COMPARISON")
print("="*70)
print(comparison)
print("="*70)

# Calculate improvements
print("\n📈 Improvement Over Baseline:")
for metric in ['Accuracy', 'F1-Score', 'ROC-AUC']:
    improvement = (comparison.loc[metric, 'Attention-BiLSTM-CNN'] - 
                   comparison.loc[metric, 'Baseline-CNN-LSTM']) * 100
    print(f"   {metric}: {improvement:+.2f}%")

## 1️⃣2️⃣ Final Summary

In [ ]:
print("\n" + "="*80)
print("🎊 PROJECT SUMMARY")
print("="*80)

print("\n📊 Dataset:")
print(f"   Total samples: {len(X)}")
print(f"   Training samples: {len(X_train_aug)}")
print(f"   Test samples: {len(X_test)}")

print("\n🏆 Best Model Performance:")
print(f"   Accuracy: {attention_metrics['Accuracy']*100:.2f}%")
print(f"   F1-Score: {attention_metrics['F1-Score']*100:.2f}%")
print(f"   ROC-AUC: {attention_metrics['ROC-AUC']*100:.2f}%")

print("\n💾 Saved Artifacts:")
print(f"   Model saved to: {MODEL_SAVE_PATH}")

print("\n🎯 Key Techniques Used:")
print("   ✅ Attention mechanism for feature focus")
print("   ✅ Bidirectional LSTM for temporal context")
print("   ✅ Skip connections for gradient flow")
print("   ✅ Batch normalization for stability")
if APPLY_AUGMENTATION:
    print("   ✅ Data augmentation for generalization")

print("\n" + "="*80)
print("✅ PROJECT COMPLETED SUCCESSFULLY!")
print("="*80 + "\n")

## 📝 Next Steps

### For Project Submission:
1. **Screenshot all visualizations** (training curves, confusion matrix, ROC curve)
2. **Save the comparison table** to Google Drive
3. **Download the trained model** from Google Drive
4. **Prepare presentation** highlighting improvements

### Optional Enhancements:
- Enable K-Fold cross-validation for more robust metrics
- Try different augmentation factors
- Experiment with hyperparameters
- Add ensemble methods

---

**Good luck with your project! 🚀**